# MERGE Y JOIN

Pandas implementa varios bloques de construcción fundamentales en la función pd.merge () y el método join () relacionado de Series y Dataframes. Como veremos, estos le permiten vincular de manera eficiente datos de diferentes fuentes.

    Categorías de combinaciones
La función pd.merge () implementa varios tipos de combinaciones: las combinaciones uno a uno, muchos a uno y muchos a muchos. Se accede a los tres tipos de combinaciones mediante una llamada idéntica a la interfaz pd.merge (); el tipo de unión realizada depende de la forma de los datos de entrada. Aquí mostraremos ejemplos simples de los tres tipos de fusiones y discutiremos opciones detalladas más adelante.

**Uniones uno a uno**
Quizás el tipo más simple de expresión de combinación es la combinación uno a uno, que en muchos aspectos es muy similar a la concatenación de columnas que se ve en Combinación de conjuntos de datos: Concat y Append. Como ejemplo concreto, considere los siguientes dos DataFrames que contienen información sobre varios empleados de una empresa:

In [4]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [5]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


Para combinar esta información en un solo DataFrame, podemos usar la función pd.merge ():

In [6]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


La función pd.merge () reconoce que cada DataFrame tiene una columna "empleado" y se une automáticamente usando esta columna como clave. El resultado de la fusión es un nuevo DataFrame que combina la información de las dos entradas. Observe que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "empleado" difiere entre df1 y df2, y la función pd.merge () lo explica correctamente. Además, tenga en cuenta que la fusión en general descarta el índice, excepto en el caso especial de las fusiones por índice (consulte las palabras clave left_index y right_index, que se comentan momentáneamente).

**Uniones de varios a uno**

Las combinaciones de varios a uno son combinaciones en las que una de las dos columnas clave contiene entradas duplicadas. Para el caso de muchos a uno, el DataFrame resultante conservará esas entradas duplicadas según corresponda. Considere el siguiente ejemplo de una combinación de varios a uno:

In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

El DataFrame resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en una o más ubicaciones según lo requieran las entradas.

**Uniones de varios a varios**

Las uniones de varios a varios son un poco confusas desde el punto de vista conceptual, pero están bien definidas. Si la columna de clave en la matriz izquierda y derecha contiene duplicados, el resultado es una combinación de muchos a muchos. Quizás esto quede más claro con un ejemplo concreto. Considere lo siguiente, donde tenemos un DataFrame que muestra una o más habilidades asociadas con un grupo en particular. Al realizar una combinación de varios a varios, podemos recuperar las habilidades asociadas con cualquier persona individual:

In [8]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Estos tres tipos de combinaciones se pueden usar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades. Pero en la práctica, los conjuntos de datos rara vez son tan limpios como con el que estamos trabajando aquí. En la siguiente sección consideraremos algunas de las opciones proporcionadas por pd.merge () que le permiten ajustar cómo funcionan las operaciones de unión.

De manera más simple, puede especificar explícitamente el nombre de la columna clave utilizando la palabra clave on, que toma un nombre de columna o una lista de nombres de columna:

In [9]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

Esta opción solo funciona si los DataFrames izquierdo y derecho tienen el nombre de columna especificado.

En ocasiones, es posible que desee fusionar dos conjuntos de datos con diferentes nombres de columna; por ejemplo, es posible que tengamos un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado". En este caso, podemos usar las palabras clave left_on y right_on para especificar los dos nombres de columna:

In [10]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, usando el método drop () de DataFrames:

In [11]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


**Especificación de la aritmética de conjuntos para combinaciones**
En todos los ejemplos anteriores, hemos pasado por alto una consideración importante al realizar una combinación: el tipo de aritmética de conjuntos utilizada en la combinación. Aparece cuando aparece un valor en una columna clave pero no en la otra. Considere este ejemplo:

In [12]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Aquí hemos fusionado dos conjuntos de datos que tienen una sola entrada de "nombre" en común: Mary. De forma predeterminada, el resultado contiene la intersección de los dos conjuntos de entradas; esto es lo que se conoce como unión interna. Podemos especificar esto explícitamente usando la palabra clave how, que por defecto es "inner":

In [13]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Otras opciones para la palabra clave cómo son 'outer', 'left' y 'right'. Una combinación externa devuelve una combinación sobre la unión de las columnas de entrada y completa todos los valores faltantes con NA:

In [14]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

El retorno de la combinación left y la combinación right se unen sobre las entradas de la izquierda y de la derecha, respectivamente. Por ejemplo:

In [15]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

Una forma de visualizar mejor este concepto es con el siguiente diagrama:


<img src="joins.png">

**Nombres de columnas superpuestos: la palabra clave sufixes**
Finalmente, puede terminar en un caso en el que sus dos DataFrames de entrada tengan nombres de columna en conflicto. Considere este ejemplo:

In [16]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

Debido a que la salida tendría dos nombres de columna en conflicto, la función de combinación agrega automáticamente un sufijo _x o _y para hacer que las columnas de salida sean únicas. Si estos valores predeterminados son inapropiados, es posible especificar un sufijo personalizado utilizando la palabra clave sufijos:

In [17]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2